In [1]:
import os
import sys
import pymeshlab
conda_active_path = sys.exec_prefix
# conda_active_path = os.environ["CONDA_PREFIX"]
sys.path.append(os.path.join(conda_active_path, "lib"))
import fileinput
import numpy as np

In [2]:
# freecad Macro - create shape and calculate deformations and save ply
import FreeCAD
import Part
import ObjectsFem
import Fem
import Mesh
import femmesh.femmesh2mesh
try:
    from femtools import ccxtools
except ImportError:
    print('ccxtools_import_passed...')
from femtools import ccxtools


ccxtools_import_passed...


In [3]:

doc = App.newDocument("doc")

box_obj = doc.addObject('Part::Box', 'Box')
box_obj.Height = 2
box_obj.Width = 5
box_obj.Length = 50

analysis_object = ObjectsFem.makeAnalysis(FreeCAD.ActiveDocument, 'Analysis')

femmesh_obj = doc.addObject('Fem::FemMeshShapeNetgenObject', 'CubeFemMesh')
femmesh_obj.Shape = box_obj 
femmesh_obj.Fineness = "VeryCoarse"
femmesh_obj.MaxSize = 100

doc.Analysis.addObject(femmesh_obj)
doc.recompute()

doc.addObject("Fem::ConstraintFixed","FemConstraintFixed")
doc.FemConstraintFixed.Scale = 1
doc.Analysis.addObject(doc.FemConstraintFixed)
for amesh in doc.Objects:
    if "FemConstraintFixed" == amesh.Name:
        print('amesh.ViewObject.Visibility = True')
    elif "Mesh" in amesh.TypeId:
        aparttoshow = amesh.Name.replace("_Mesh","")
        for apart in doc.Objects:
            if aparttoshow == apart.Name:
                print('apart.ViewObject.Visibility = True')
        print('amesh.ViewObject.Visibility = False')

doc.recompute()

doc.FemConstraintFixed.Scale = 1
doc.FemConstraintFixed.References = [(doc.Box,"Face2")]
doc.recompute()

doc.addObject("Fem::ConstraintForce","FemConstraintForce")
doc.FemConstraintForce.Force = 1.0
doc.FemConstraintForce.Reversed = False
doc.FemConstraintForce.Scale = 1
doc.Analysis.addObject(doc.FemConstraintForce)

for amesh in doc.Objects:
    if "FemConstraintForce" == amesh.Name:
        print("amesh.ViewObject.Visibility = True")
    elif "Mesh" in amesh.TypeId:
        aparttoshow = amesh.Name.replace("_Mesh","")
        for apart in doc.Objects:
            if aparttoshow == apart.Name:
                print("apart.ViewObject.Visibility = True")
        print("amesh.ViewObject.Visibility = False")

doc.recompute()
doc.FemConstraintForce.Force = 1000
doc.FemConstraintForce.Direction = None
doc.FemConstraintForce.Reversed = True
doc.FemConstraintForce.Scale = 1
doc.FemConstraintForce.References = [(doc.Box,"Face6")]
doc.recompute()

analysis_object.addObject(ObjectsFem.makeSolverCalculixCcxTools(doc))

## material
material_object = ObjectsFem.makeMaterialSolid(doc, "SolidMaterial")
mat = material_object.Material
mat['Name'] = "Steel-Generic"
mat['YoungsModulus'] = "210000 MPa"
mat['PoissonRatio'] = "0.30"
mat['Density'] = "7900 kg/m^3"
material_object.Material = mat
analysis_object.addObject(material_object)

# run the analysis step by step
fea1 = ccxtools.FemToolsCcx()
fea1.update_objects()
fea1.setup_working_dir()

fea1.setup_ccx()
message = fea1.check_prerequisites()
if not message:
    fea1.purge_results()
    fea1.write_inp_file()
    fea1.ccx_run()
    print(f'+o+o+o+o+o+o+ .inp filename {fea1.inp_file_name}')
    fea1.load_results()
else:
    FreeCAD.Console.PrintError("Oh, we have a problem! {}\n".format(message))  # in report view
    print("Oh, we have a problem! {}\n".format(message))  # in python console

femmesh_obj = doc.getObject("ResultMesh").FemMesh
result = doc.getObject("CCX_Results")

out_mesh = femmesh.femmesh2mesh.femmesh_2_mesh(femmesh_obj, result)

for o in doc.Objects:
    print(o.Name)

apart.ViewObject.Visibility = True
amesh.ViewObject.Visibility = False
amesh.ViewObject.Visibility = True
apart.ViewObject.Visibility = True
amesh.ViewObject.Visibility = False
amesh.ViewObject.Visibility = True
+o+o+o+o+o+o+ .inp filename /tmp/femsdjpev9b/CubeFemMesh.inp
found a last Face:  167125904785538
CCX_Results
Mesh by surface search method:  0.369124
Box
Analysis
CubeFemMesh
FemConstraintFixed
FemConstraintForce
CalculiXccxTools
SolidMaterial
ResultMesh
CCX_Results


In [4]:
# calculate  displacements and add updated femmesh to the analysis (remove the old one)
mesh_obj = doc.CubeFemMesh
#remove the dold femmesh
doc.removeObject('CubeFemMesh')
#add updated femmesh - with the same name (->>> update old femmesh)
mesh_obj = doc.addObject('Fem::FemMeshObject', 'CubeFemMesh')

new_mesh = Fem.FemMesh()
res_coord = doc.CCX_Results.DisplacementVectors
res_nd_no = doc.CCX_Results.NodeNumbers
res_mesh = doc.ResultMesh.FemMesh
for n in res_nd_no:
    new_mesh.addNode((res_mesh.Nodes[n].x + res_coord[n-1].x), (res_mesh.Nodes[n].y + res_coord[n-1].y), (res_mesh.Nodes[n].z + res_coord[n-1].z), n)

for v in res_mesh.Volumes:
    new_mesh.addVolume( list(res_mesh.getElementNodes(v)))

mesh_obj.FemMesh = new_mesh
doc.Analysis.addObject(mesh_obj)

[<Fem::FemMeshObject object>]

In [5]:
for o in doc.Objects:
    print(o.Name)

Box
Analysis
FemConstraintFixed
FemConstraintForce
CalculiXccxTools
SolidMaterial
ResultMesh
CCX_Results
CubeFemMesh


In [6]:
# update the last inp file (nodes and elements): 
fea2 = ccxtools.FemToolsCcx()#
fea2.update_objects()#

fea2.inp_file_name = fea1.inp_file_name#
wd = fea2.inp_file_name.replace('CubeFemMesh.inp','')
fea2.working_dir = wd
message = fea2.check_prerequisites()#
print(message)

print(f'+o+o+o+o+o+o+ .inp filename {fea2.inp_file_name}')
if not message:
    fea2.reset_all()#
    fea2.ccx_run()
    fea2.load_results()#
    print('ccx run done!')
else:
    print("Oh, we have a problem! {}\n".format(message))  # in python console

print(f'+o+o+o+o+o+o+ .inp filename {fea2.inp_file_name}')


+o+o+o+o+o+o+ .inp filename /tmp/femsdjpev9b/CubeFemMesh.inp
ccx run done!
+o+o+o+o+o+o+ .inp filename /tmp/femsdjpev9b/CubeFemMesh.inp


In [7]:
inp_dir, inpfile = os.path.split(fea2.inp_file_name)
inp_dir

'/tmp/femsdjpev9b'

In [8]:
# new action - could be removed --> directly based on femmesh 
doc.recompute()
doc.FemConstraintForce.Force = 10
doc.FemConstraintForce.Direction = None
doc.FemConstraintForce.Reversed = True
doc.FemConstraintForce.Scale = 1
doc.FemConstraintForce.References = [(doc.Box,"Face5")]
doc.recompute()

# prepare force constraint for inp file format
frc = "{:.13E}".format(doc.FemConstraintForce.Force)
Fx = np.random.randint(1, 50)
Fy = np.random.randint(1,5)
# print(Fx,Fy)

with open(os.path.join(inp_dir,"ConstForce.txt"), "w") as text_file_force , open(os.path.join(inp_dir,"ConstFix.txt"), "w") as text_file_fixed:
    for it in doc.CubeFemMesh.FemMesh.Nodes.items():
        if np.ceil(it[1].x) == Fx and np.ceil(it[1].y) == Fy and it[1].z!=0:      # Fx , Fy placement, check z (to remove bottom surface)
            # print(it[0],',3,',frc)
            write_str = f'{it[0]},3,{frc}\n'
            text_file_force.write(write_str)
        if it[1].z==0: # check z(bottom surface)
            # print(it[0],',\n')
            write_str = f'{it[0]},\n'
            text_file_fixed.write(write_str)
    
    text_file_force.write('\n\n***********************************************************')#end of block    
    text_file_fixed.write('\n\n***********************************************************')#end of block

update inp file (insert new CLOAD,... ) 

In [9]:
def create_head_tail_files(fl_dir,inpfile,HEAD,TAIL): # HEAD:included, TAIL:excluded
    ln = 0
    start_line = 0
    end_line = 0

    with open(os.path.join(fl_dir,inpfile),"r") as fin, open(os.path.join(fl_dir,'head.txt'),"w") as fout_head, open(os.path.join(fl_dir,'tail.txt'),"w") as fout_tail:
        for line in fin:
            ln = ln+1
            if start_line==0:
                fout_head.write(line)
                if HEAD in line:
                    start_line = ln
            
            if TAIL in line: #TAIL is excluded
                    end_line = ln-1
            
            if end_line!=0:
                fout_tail.write(line)
                
                
def insert_block_to_inp(inp_dir,inpfile,BLOCK):
    file_list = ['head.txt', BLOCK, 'tail.txt']
    with open(inpfile, 'w') as file:
        input_lines = fileinput.input(file_list)
        file.writelines(input_lines)    


In [10]:
#CLOAD
HEAD = '** FemConstraintForce'
TAIL = '** Outputs --> frd file'
BLOCK = 'ConstForce.txt'                
create_head_tail_files(inp_dir,inpfile,HEAD,TAIL)

cwd = os.getcwd()
os.chdir(inp_dir)       
insert_block_to_inp(inp_dir,inpfile,BLOCK)    
os.chdir(cwd)

In [11]:
### HEAD and TAIL are valid for one standard inp (the same sequence of blocks...)
HEAD = '*NSET,NSET=FemConstraintFixed'
TAIL = '** Materials'
BLOCK = 'ConstFix.txt'
create_head_tail_files(inp_dir,inpfile,HEAD,TAIL)

cwd = os.getcwd()
os.chdir(inp_dir)
insert_block_to_inp(inp_dir,inpfile,BLOCK)    
os.chdir(cwd)

In [12]:

# frc = "{:.13E}".format(doc.FemConstraintForce.Force)
# Fx = np.random.randint(1, 50)
# Fy = np.random.randint(1,5)
# print(Fx,Fy)

# for it in doc.CubeFemMesh.FemMesh.Nodes.items():
#     if np.ceil(it[1].x) == Fx and np.ceil(it[1].y) == Fy and it[1].z!=0:      # Fx , Fy placement, check z (to remove bottom surface)
#         print(it[0],',3,',frc)
